In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import string
import datetime
import re
import tqdm

In [3]:
from collections import namedtuple
from gensim.models import doc2vec
from konlpy.tag import Twitter
import multiprocessing
from pprint import pprint
from gensim.models import Doc2Vec
from sklearn.linear_model import LogisticRegression
import numpy
import pickle

## 네이버 기사 크롤링(201601 ~ 201804)

In [101]:
URL = 'http://news.naver.com/main/hotissue/dailyList.nhn?'

CATEGORY = { '100' : '정치',
             '101' : '경제',
             '102' : '사회',
             '103' : '생활/문화',
             '104' : '세계',
             '105' : 'it/과학' }

OID = 999
MAX_AID = 100000
S_YEAR = 2016
S_MONTH = 1
S_DAY = 1
ENDATE = { 'year':2019,
            'month':13,
            'day':31
           }

def clawler_loop():
    for _year in range(S_YEAR, ENDATE['year']):
        if _year == 2018:
            ENDATE['month'] = 5
        for _month in range(S_MONTH, ENDATE['month']):
            print(str(_year) + str(_month).zfill(2) + " START")
            for _day in range(S_DAY, ENDATE['day']):
                for _list in CATEGORY:
                    index_date = URL + 'sid1='+  _list +'&date='+ str(_year) + str(_month).zfill(2) + str(_day).zfill(2)
                    date_time = str(_year) + str(_month).zfill(2)
                    naver_sid_write(date_time, index_date, './data/crawler.txt','a')
            sleep(0.01)


def naver_sid_write(date_time, _url, _file_name, _option):
    i = 0
    _response = requests.get(_url)
    _plain_text = _response.text
    
    _soup = BeautifulSoup(_plain_text, 'lxml')

    _lists = _soup.find('div', class_='list_body ranking_body')
    replace_str = str(_lists).replace('</a>', '\n')
    _split_list = replace_str.split('\n')

    _file = open(_file_name, _option)

    for _list in _split_list:
        if _list.find('href') != -1:
            i += 1
            _start = _list.find('sid1=')
            _category =_list[_start+5:_start+8]
            f_write_str = CATEGORY[_category] + "\t" + _list[114:] + "\t" + str(i) +  "\n"
            _file.write(f_write_str) 

    _file.close()
        

def main():
    clawler_loop()

if __name__ == "__main__":
    main()

201601 START
201602 START
201603 START
201604 START
201605 START
201606 START
201607 START
201608 START
201609 START
201610 START
201611 START
201612 START
201701 START
201702 START
201703 START
201704 START
201705 START
201706 START
201707 START
201708 START
201709 START
201710 START
201711 START
201712 START
201801 START
201802 START
201803 START
201804 START


## 네이버 영화 댓글 크롤링(어벤져스: 인피니티 워)

In [56]:
URL = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=136315&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='

def clawler_loop(URL):
    for i in range(2172):
        clawler_page = URL + str(i)
        naver_movie_comment(clawler_page, './data/moviecommentcrawler.txt', 'a')
        if i % 100 == 0:
            print('page : ' + str(i))
        sleep(0.01)

def naver_movie_comment(URL, _file_name, _option):
    i = 0
    _response = requests.get(URL)
    _plain_text = _response.text

    _soup = BeautifulSoup(_plain_text, 'lxml')
    _lists = _soup.find('div', class_='score_result')

    replace_str = str(_lists).replace('</p>', '\n')
    _split_list = replace_str.split('\n')
    
    _file = open(_file_name, _option)
    
    for _list in _split_list:
        if _list.find("<p>") == 0:
            i += 1
            comment = _list.replace('<span class="ico_viewer">관람객</span>','').replace('<p>','')
            comment = comment + "\t" + str(i) +  "\n"
            _file.write(comment)
            
    _file.close()
    
clawler_loop(URL)


page : 0
page : 100
page : 200
page : 300
page : 400
page : 500
page : 600
page : 700
page : 800
page : 900
page : 1000
page : 1100
page : 1200
page : 1300
page : 1400
page : 1500
page : 1600
page : 1700
page : 1800
page : 1900
page : 2000
page : 2100


## 영화 평점 데이터를 가지고 긍정, 부정 트래이닝

In [25]:
#doc2vec parameters
cores = multiprocessing.cpu_count()

vector_size = 300
window_size = 15
word_min_count = 2
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 1
worker_count = cores


# 트래이닝 데이터 읽기
train_data = read_data('./data/ratings_train.txt')

# 형태소 분류
train_docs = [(tokenize(row[1]), row[2]) for row in train_data[1:]]

# doc2vec 에서 필요한 데이터 형식으로 변경
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]

# 사전 구축
doc_vectorizer = doc2vec.Doc2Vec(size=300, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)

# Train document vectors!
for epoch in range(10):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002  # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

#To save
doc_vectorizer.save('./model/doc2vec.model')

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


In [24]:
twitter = Twitter()

def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
    return data

def tokenize(doc):
  # norm, stem은 optional
  return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]


# 테스트 데이터를 읽기
train_data = read_data('./data/ratings_train.txt')
test_data = read_data('./data/ratings_test.txt')

# 형태소 분류
train_docs = [(tokenize(row[1]), row[2]) for row in train_data[1:]]
test_docs = [(tokenize(row[1]), row[2]) for row in test_data[1:]]

# doc2vec 에서 필요한 데이터 형식으로 변경
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]
tagged_test_docs = [TaggedDocument(d, [c]) for d, c in test_docs]

# load train data
doc_vectorizer = Doc2Vec.load('./model/doc2vec.model')

# 분류를 위한 피쳐 생성
train_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]
train_y = [doc.tags[0] for doc in tagged_train_docs]
test_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_test_docs]
test_y = [doc.tags[0] for doc in tagged_test_docs]


#classifier = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)
classifier = LogisticRegression(random_state=1234)
classifier.fit(train_x, train_y)

# 테스트 socre 확인
print( classifier.score(test_x, test_y) )
# 0.63904

# save the model to disk
filename = './model/finalized_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

0.63816


## 크롤링한 네이버 기사를 가지고 긍정 부정 분류

In [51]:
# 실제 구동 데이터를 읽기
run_data = read_data('./data/crawler.txt')

# 형태소 분류
run_docs = [(tokenize(row[1]), row[2]) for row in run_data[:]]

# doc2vec 에서 필요한 데이터 형식으로 변경
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_run_docs = [TaggedDocument(d, [c]) for d, c in run_docs]

# load train data
doc_vectorizer = Doc2Vec.load('./model/doc2vec.model')

# 분류를 위한 피쳐 생성
run_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_run_docs]
run_y = [doc.tags[0] for doc in tagged_run_docs]

# load the model from disk
filename = './model/finalized_model.sav'

# 실제 분류 확인
loaded_model = pickle.load(open(filename, 'rb'))
print(loaded_model.predict(run_x[0].reshape(1, -1)))
print(loaded_model.predict(run_x[1].reshape(1, -1)))

['1']
['0']


In [53]:
_file = open('./data/result.txt', 'a')
for i in range(len(run_data)):
    data = run_data[i][1]
    f_write_str = data + "\t" +loaded_model.predict(run_x[i].reshape(1, -1))[0] + "\n"
    _file.write(f_write_str) 
_file.close()

## 기사 타이틀을 긍정,부정 분류한 텍스트 파일

In [54]:
result_data = read_data('./data/result.txt')
result_data

[["野 텃밭 호남서 더민주·安신당 '엎치락 뒤치락'", '1'],
 ['北 외무성, 한일 위안부 합의 "용납될 수 없다" 비난', '1'],
 ['차기 대선주자 선호도 1위 반기문…방송사 신년 여론조사', '0'],
 ['구름 위에서, 파도 헤치고…“대한민국 철통 수호”', '1'],
 ['김정은, 신년사서 ‘핵’ 언급 않고 대화 시사', '', '0'],
 ['[단독]선거구 획정위 “지역구 7석 확대안 제출”', '0'],
 ['평양지하철, 전광판 설치된 신형 전동차 운행 개시', '1'],
 ['[여론조사] ① 대선주자 지지도 1위는?', '1'],
 ['“안철수 신당 급부상…더민주와 접전”', '1'],
 ['평양서 새해 불꽃놀이 열려…예고보다 21시간 늦어', '1'],
 ['[여론조사] ② 경제·사회, 민심은?', '1'],
 ['반 총장 “박 대통령, 위안부 합의 올바른 용단”', '1'],
 ['선거구 획정 직권상정 절차 돌입…여야 반발', '1'],
 ['새해 첫 초계비행…‘영공 수호’ 다짐', '1'],
 ['합참의장, 새해 첫 해상 지휘…서해 ‘이상 무’', '1'],
 ['北 김정은, 대화 의지·南 비난 동시 언급', '0'],
 ['‘누리과정 예산’ 놓고 경기도의회 몸싸움', '0'],
 ['북한의 새해맞이', '0'],
 ['北외무성 "日, 일본군 성노예 범죄 사죄배상해야"', '1'],
 ["'누리예산' 놓고 볼썽사나운 몸싸움…4명 병원행", '1'],
 ['北 김정은 2016년 신년사, 분야별 주요 내용은 뭔가(종합)', '1'],
 ['朴 대통령 "대화 門 항상 열어"…김정은 "허심탄회 대화"', '0'],
 ['김정은 집권 5년째, 올 한해 남북관계 쟁점은?', '1'],
 ['꿈틀대는 민심…신년 여론조사로 본 정치권 전망은', '1'],
 ["경기도의회 '누리 예산' 놓고 몸싸움, 예산안 처리 무산", '0'],
 ['문재인-안철수, 신년부터 경쟁적 행보…야권재편 가속', '1'],
 ["北 김정은 신년사, '핵' 대신 대화의

## 크롤링한 영화 댓글을 가지고 긍정 부정 분류

In [57]:
# 실제 구동 데이터를 읽기
run_comment_data = read_data('./data/moviecommentcrawler.txt')

# 형태소 분류
run_comment_docs = [(tokenize(row[0]), row[1]) for row in run_comment_data[:]]

# doc2vec 에서 필요한 데이터 형식으로 변경
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_run_comment_docs = [TaggedDocument(d, [c]) for d, c in run_comment_docs]

# load train data
doc_vectorizer = Doc2Vec.load('./model/doc2vec.model')

# 분류를 위한 피쳐 생성
run_x = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_run_comment_docs]
run_y = [doc.tags[0] for doc in tagged_run_comment_docs]

# load the model from disk
filename = './model/finalized_model.sav'

# 실제 분류 확인
loaded_model = pickle.load(open(filename, 'rb'))
print(loaded_model.predict(run_x[0].reshape(1, -1)))
print(loaded_model.predict(run_x[1].reshape(1, -1)))

['0']
['1']


In [58]:
_file = open('./data/comment_result.txt', 'a')
for i in range(len(run_comment_data)):
    data = run_comment_data[i][0]
    f_write_str = data + "\t" +loaded_model.predict(run_x[i].reshape(1, -1))[0] + "\n"
    _file.write(f_write_str) 
_file.close()

## 영화 댓글을 긍정,부정 분류한 텍스트 파일

In [59]:
comment_result_data = read_data('./data/comment_result.txt')
comment_result_data

[['닥터 스트레인지 마지막 대사 "이제 다 끝났어"는 오역입니다. "이게 마지막 단계야"가 옳은 번역입니다. 모두들 볼 수 있게 위로 올려주세요. 또 마블의 번역가 박읍읍씨가 해냈네요. 영화 결말을 바꿔버리다니  ',
  '0'],
 ['번역가좀 짤라주세요 제발  ', '1'],
 ['그렇다. 토르는 신이엿다...  ', '1'],
 ['그냥 너무 충격먹어서 할말이없다...  ', '0'],
 ['모든건 닥터가 알고있다!!타임스톤 타노스한테 쉽게준건 미래를 봤거든...몇만가지중 한가지방법에 모든걸 걸었음!!!  ', '1'],
 ['내년까지 살아가야 할 이유가 생겼습니다  ', '0'],
 ['기존의 마블 영화를 생각하고 본다면 결말에 상당히 충격을 먹게 될 것이다.  ', '1'],
 ['영화는 너무 좋은데 자막 돌았나? 닥터 스트레인지가 마지막에 이젠 다 끝났어가 아니라 이제 최종단계야가 맞는 해석아님? 글고 쿠키영상에 마더파커를 어머니찾는 효자로 만들어놨네ㄷㄷ 영화의 재미를 자막이 반감시키는듯 하다ㅠ  ',
  '1'],
 ['3시간짜리 어벤져스4  예고편보고옴  ', '1'],
 ['토르 지구 귀환 씬에서 탄성이 절로 나옴  ', '0'],
 ['성지예약)) 잘봐라 닥터스트레인지가 첨에 분명 느그들 다 죽어도 타임스톤 지킨다고 했지 글구나서 이기는 경우 1개라했고 근데 토니 죽기직전에 타임스톤을 넘김 이건, 타임스톤을 넘겨야만 승리한다는것. 여기서 예측=타노스가 가모라를 위해 시간을 돌릴것임.  ',
  '1'],
 ['마블한테 댓글알바랜다 ㅋㅋㅋ마이클잭슨이 음반사재기 하는소리하네  ', '1'],
 ['유튜버 추측한 인간들 다 틀렸어 ㅋㅋㅋ  ', '1'],
 ['타노스에게 당하고도 주늑들지 않았던것은 토르뿐이였다영화마지막 4살짜리 아이처럼신을찾는 캡틴과 스타크의 좌절..결국 그들도 영웅이기 이전에 재앙과 전쟁이 두려운 그저 인간이었다는것 누가쎄다 약하다보다 인간과 신의차이를 보여준영화 토르는 신이였다..  ',
  '0'],
 ['이 영화의 최고의

```
어느 정도 분류 된거 같긴 한데 제대로 되지 않은거 같음 향후 과제로 좀더 다양한 감정을 학습 시켜서 다양한 감정을 표현 하게끔 만드는 것을 목표로 해야 겠다.
```